In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver import ActionChains
from selenium.webdriver.common.alert import Alert

#criar navegador
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)    #inicializa o navegador

#abrir a página index (entrar no site da busca jurídica)
import os
caminho = os.getcwd() #pasta que está o código
arquivo = caminho + r"\index.html"

In [2]:
#Importar base de dados

import pandas as pd

tabela_processos = pd.read_excel('Processos.xlsx')
display(tabela_processos)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [3]:
for linha in tabela_processos.index: 

    #para cada processo (linha da tabela)
    navegador.get(arquivo)
    
    #abrir a lista de cidades
    botao = navegador.find_element(By.XPATH, '/html/body/div/div/button')
    ActionChains(navegador).move_to_element(botao).perform()

    cidade = tabela_processos.loc[linha, 'Cidade']

    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

    #Mudar para a nova aba
    aba_original = navegador.window_handles[0]
    indice = linha + 1
    nova_aba = navegador.window_handles[indice]

    navegador.switch_to.window(nova_aba)

    #Preencher o formulário com os dados de busca

    #nome da parte
    navegador.find_element(By.NAME, 'nome').send_keys(tabela_processos.loc[linha, 'Nome'])
    #advogado
    navegador.find_element(By.ID, 'advogado').send_keys(tabela_processos.loc[linha, 'Advogado'])
    #número do processo
    navegador.find_element(By.ID, 'numero').send_keys(tabela_processos.loc[linha, 'Cidade'])
    
    #Clicar em pesquisar
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

    #Confirmar pesquisa
    #from selenium.webdriver.common.alert import Alert
    alerta = Alert(navegador)
    alerta.accept()

    #Esperar o resultado da pesquisa e agir de acordo com o resultado
    import time
    
    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)
    texto_alerta = alerta.text
    
    if "Processo encontrado com sucesso" in texto_alerta:
        alerta.accept()
        tabela_processos.loc[linha, 'Status'] = 'Processo encontrado'
    else:
        alerta.accept()
        tabela_processos.loc[linha, 'Status'] = 'Processo não encontrado'

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_5972\1757298082.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Processo não encontrado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tabela_processos.loc[linha, 'Status'] = 'Processo não encontrado'


In [4]:
navegador.quit()
display(tabela_processos)

tabela_processos.to_excel("Processos Atualizado.xlsx")

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Processo não encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Processo encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Processo não encontrado
3,Carol,Amanda,PC5197,São Paulo,Processo encontrado
